<a href="https://colab.research.google.com/github/dev0419/DL_Lab/blob/main/RNN_l8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Dataset contains Monthly and Daily prices of Natural gas, starting from January 1997 to current year. Prices are in nominal dollars. The task is to predict the price of natural gas using RNN model for the dataset
https://datahub.io/core/natural-gas#resource-daily or

https://www.kaggle.com/datasets/joebeachcapital/natural-gas-prices Given the price of last ten 10 days, corresponds to sequence_length, the RNN model must predict the price for the 11th day.

In [1]:
#!wget  https://datahub.io/core/natural-gas#resource-daily
#!wget  https://www.kaggle.com/datasets/joebeachcapital/natural-gas-prices
#!pip install kaggle

In [2]:
from google.colab import files
#uploaded = files.upload()

In [3]:
'''!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json'''


'!mkdir ~/.kaggle\n!mv kaggle.json ~/.kaggle/\n!chmod 600 ~/.kaggle/kaggle.json'

In [4]:
#!kaggle datasets download -d joebeachcapital/natural-gas-prices

In [5]:
#!unzip natural-gas-prices.zip

In [6]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [7]:
df = pd.read_csv('/content/daily_csv.csv')
df.head()

,Date,Price
0,1997-01-07,3.82
1,1997-01-08,3.80
2,1997-01-09,3.61
3,1997-01-10,3.92
4,1997-01-13,4.00


In [8]:
df.isna().value_counts()

Date   Price
False  False    5952
       True        1
dtype: int64

In [9]:
df.dropna(inplace=True)
df.isna().value_counts()

Date   Price
False  False    5952
dtype: int64

In [10]:
y = df['Price'].values
x = np.arange(1,len(y),1)
print(x)
print(y)
len(y)

[   1    2    3 ... 5949 5950 5951]
[3.82 3.8  3.61 ... 2.46 2.3  2.22]


5952

In [11]:
scaler = MinMaxScaler()
y = scaler.fit_transform(y.reshape(-1,1))
y = y.flatten()
y.shape

(5952,)

In [13]:
Sequence_Length = 10
X = []
Y = []

for i in range(0, len(y) - Sequence_Length):
    list1 = []
    for j in range(i, i + Sequence_Length):
        list1.append(y[j])
    X.append(list1)
    Y.append(y[i + Sequence_Length])

In [17]:
print(X)
print(Y)

[[0.1589213998852553, 0.15777395295467583, 0.14687320711417096, 0.1646586345381526, 0.16924842226047046, 0.16982214572576018, 0.18875502008032127, 0.2099827882960413, 0.16408491107286288, 0.1267928858290304], [0.15777395295467583, 0.14687320711417096, 0.1646586345381526, 0.16924842226047046, 0.16982214572576018, 0.18875502008032127, 0.2099827882960413, 0.16408491107286288, 0.1267928858290304, 0.11130235226620772], [0.14687320711417096, 0.1646586345381526, 0.16924842226047046, 0.16982214572576018, 0.18875502008032127, 0.2099827882960413, 0.16408491107286288, 0.1267928858290304, 0.11130235226620772, 0.11474469305794607], [0.1646586345381526, 0.16924842226047046, 0.16982214572576018, 0.18875502008032127, 0.2099827882960413, 0.16408491107286288, 0.1267928858290304, 0.11130235226620772, 0.11474469305794607, 0.1095811818703385], [0.16924842226047046, 0.16982214572576018, 0.18875502008032127, 0.2099827882960413, 0.16408491107286288, 0.1267928858290304, 0.11130235226620772, 0.11474469305794607

In [18]:
X = np.array(X)
Y = np.array(Y)

In [19]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.10,random_state=42,shuffle=False,stratify=None)

In [23]:
class NGTTimeSeries(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x,dtype=torch.float32)
        self.y = torch.tensor(y,dtype=torch.float32)

    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]

    def __len__(self):
        return len(self.x)

In [24]:
dataset = NGTTimeSeries(X_train,Y_train)
train_loader = DataLoader(dataset,batch_size=256,shuffle=True)

In [ ]:
class RNNModel(nn.Module):
    def __init__(self):
        super().__init__()
